<a href="https://colab.research.google.com/github/MAZanabria/InteligenciaArtificialZVMA/blob/pacman/reg_log_onevsall_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Clasificación multiclase

dataset de clasificacion de generos musicales

Contenido
Conjunto de datos de entrenamiento: 17.996 filas con 15 columnas

Detalles de la columna: nombre del artista; nombre de la pista; popularidad; 'bailabilidad'; energía; llave; volumen; modo; 'habla'; 'acústica'; 'instrumentalidad'; vivacidad; valencia; tempo; duración en milisegundos y firma_hora.

Variable objetivo: 'Clase' como Rock, Indie, Alt, Pop, Metal, HipHop, Alt_Music, Blues, Acústica/Folk, Instrumental, Country, Bollywood,

Conjunto de datos de prueba: 7713 filas con 14 columnas

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# utilizado para la manipulación de directorios y rutas
import os

# Cálculo científico y vectorial para python
import numpy as np

# Libreria para graficos
from matplotlib import pyplot

# Modulo de optimizacion en scipy
from scipy import optimize

# le dice a matplotlib que incruste gráficos en el cuaderno
%matplotlib inline

## 1 Clasificación multiclase

La primera parte del ejercicio, extenderá la implementación anterior de la regresión logística y la aplicará a la clasificación de uno contra todos (one vs all).

Se inicia el ejercicio cargando primero el conjunto de datos.

In [40]:
import pandas as pd
# La entrada es de 14 elemento contando con x0
input_layer_size  = 14

# 10 etiquetas, de 1 a 10 (tomar en cuenta que se asigna "0" a la etiqueta 10)
num_labels = 10

# Cargar los datos de entrenamiento y prueba
data_train = pd.read_excel('/content/drive/MyDrive/sis 420/Laboratorio 3/prueba.xlsx')
data_test = pd.read_excel('/content/drive/MyDrive/sis 420/Laboratorio 3/test11.xlsx')

X_train = data_train.drop(columns=[data_train.columns[14]])  # Excluir la columna 14 como características
y_train = data_train.iloc[:, 14]   # Seleccionar la columna 14 como variable objetivo

X_test = num_labels

m_train = y_train.size  # Obtener el tamaño del conjunto de entrenamiento

print(X_train.head())  # Imprimir las primeras filas de las características
print(y_train)  # Imprimir la variable objetivo



   60.0  0.854  0.564   1.0     -4964    1  0.0485    0.0171         0  \
0  54.0  0.382  0.814   3.0    -7.230  1.0  0.0406  0.001100  0.004010   
1  35.0  0.434  0.614   6.0 -8334.000  1.0  0.0525  0.486000  0.000196   
2  66.0  0.853  0.597  10.0 -6528.000  0.0  0.0555  0.021200  0.000000   
3  53.0  0.167  0.975   2.0 -4279.000  1.0  0.2160  0.000169  0.016100   
4  53.0  0.235  0.977   6.0     0.878  1.0  0.1070  0.003530  0.006040   

   0.0849   0.899     134071  234596.0    4  
0   0.101  0.5690  116454.00  251733.0  4.0  
1   0.394  0.7870  147681.00  109667.0  4.0  
2   0.122  0.5690  107033.00  173968.0  4.0  
3   0.172  0.0918     199.06  229960.0  4.0  
4   0.172  0.2410  152952.00  208133.0  4.0  
0        10.0
1         6.0
2         5.0
3        10.0
4         6.0
         ... 
17987     6.0
17988     2.0
17989     8.0
17990     8.0
17991    10.0
Name: 5, Length: 17992, dtype: float64


In [41]:
print(data.head())  # Muestra las primeras filas del DataFrame cargado


   60.0  0.854  0.564   1.0     -4964    1  0.0485    0.0171         0  \
0  54.0  0.382  0.814   3.0    -7.230  1.0  0.0406  0.001100  0.004010   
1  35.0  0.434  0.614   6.0 -8334.000  1.0  0.0525  0.486000  0.000196   
2  66.0  0.853  0.597  10.0 -6528.000  0.0  0.0555  0.021200  0.000000   
3  53.0  0.167  0.975   2.0 -4279.000  1.0  0.2160  0.000169  0.016100   
4  53.0  0.235  0.977   6.0     0.878  1.0  0.1070  0.003530  0.006040   

   0.0849   0.899     134071  234596.0    4    5  
0   0.101  0.5690  116454.00  251733.0  4.0  0.0  
1   0.394  0.7870  147681.00  109667.0  4.0  6.0  
2   0.122  0.5690  107033.00  173968.0  4.0  5.0  
3   0.172  0.0918     199.06  229960.0  4.0  0.0  
4   0.172  0.2410  152952.00  208133.0  4.0  6.0  


Normalizamos los datos

In [42]:
import numpy as np

def featureNormalize(X):
    X_norm = X.copy()
    mu = np.zeros(X.shape[1])
    sigma = np.zeros(X.shape[1])

    mu = np.mean(X, axis=0)
    sigma = np.std(X, axis=0)
    X_norm = (X - mu) / sigma

    return X_norm, mu, sigma

# Llama a featureNormalize con los datos cargados
X_norm, mu, sigma = featureNormalize(X_train)

# Imprime las características normalizadas, las medias y las desviaciones estándar
print("Características normalizadas:")
print(X_norm)
print("\nMedias:")
print(mu)
print("\nDesviaciones estándar:")
print(sigma)


Características normalizadas:
           60.0     0.854     0.564       1.0     -4964         1    0.0485  \
0      0.546711 -0.320296  0.642228 -0.013805  1.568385  0.016235 -0.464428   
1     -0.537008 -0.220311 -0.207571  0.008297 -0.273176  0.016235 -0.323935   
2      1.231166  0.585337 -0.279804  0.037767  0.126241  0.007025 -0.288517   
3      0.489673 -0.733695  1.326317 -0.021173  0.623633  0.016235  1.606367   
4      0.489673 -0.602946  1.334815  0.008297  1.570178  0.016235  0.319499   
...         ...       ...       ...       ...       ...       ...       ...   
17987 -0.537008 -0.735618 -2.353315  0.015664  1.566202  0.007025 -0.456164   
17988 -0.993312  0.171938 -1.868930  0.045134 -0.680114  0.007025 -0.555335   
17989 -0.594046  0.018115  1.351811 -0.006438  0.534284  0.007025 -0.103160   
17990 -0.879236 -0.641401  0.603987  0.008297 -1.251374  0.007025  0.638264   
17991 -0.080705 -0.285686  0.807939 -0.006438  1.568807  0.007025 -0.246015   

         0.0171      

In [43]:
import numpy as np

# Configurar la matriz adecuadamente, y agregar una columna de unos que corresponde al término de intercepción.
m, n = X_norm.shape
# Agregar el término de intercepción a X
X = np.concatenate([np.ones((m, 1)), X_norm], axis=1)


### 1.2 Visualización de los datos


In [44]:
# def displayData(X, example_width=None, figsize=(10, 10)):
#     """
#     Muestra datos 2D almacenados en X en una cuadrícula apropiada.
#     """
#     # Calcula filas, columnas
#     if X.ndim == 2:
#         m, n = X.shape
#     elif X.ndim == 1:
#         n = X.size
#         m = 1
#         X = X[None]  # Promocionar a una matriz bidimensional
#     else:
#         raise IndexError('La entrada X debe ser 1 o 2 dimensinal.')

#     example_width = example_width or int(np.round(np.sqrt(n)))
#     example_height = n / example_width

#     # Calcula el numero de elementos a mostrar
#     display_rows = int(np.floor(np.sqrt(m)))
#     display_cols = int(np.ceil(m / display_rows))

#     fig, ax_array = pyplot.subplots(display_rows, display_cols, figsize=figsize)
#     fig.subplots_adjust(wspace=0.025, hspace=0.025)

#     ax_array = [ax_array] if m == 1 else ax_array.ravel()

#     for i, ax in enumerate(ax_array):
#         ax.imshow(X[i].reshape(example_width, example_width, order='F'),
#                   cmap='Greys', extent=[0, 1, 0, 1])
#         ax.axis('off')

In [45]:
# Selecciona aleatoriamente 100 puntos de datos para mostrar
rand_indices = np.random.choice(m, 100, replace=False)
sel = X[rand_indices, :]

# displayData(sel)

In [46]:
#mi codigo con mis datos
import numpy as np

# Configurar la matriz adecuadamente, y agregar una columna de unos que corresponde al término de intercepción.
m, n = X_norm.shape
# Agregar el término de intercepción a X
X = np.concatenate([np.ones((m, 1)), X_norm], axis=1)


### 1.3 Vectorización de regresión logística

Se utilizará múltiples modelos de regresión logística uno contra todos para construir un clasificador de clases múltiples. Dado que hay 10 clases, deberá entrenar 10 clasificadores de regresión logística separados.

<a id="section1"></a>
#### 1.3.1 Vectorización de la funcion de costo


#### 1.3.2 Vectorización del gradiente



In [48]:
def sigmoid(z):
    """
    Calcula la sigmoide de z.
    """
    return 1.0 / (1.0 + np.exp(-z))

In [49]:

import numpy as np

def lrCostFunction(theta, X, y, lambda_):
    """
    Calcula el costo de usar theta como parámetro para la regresión logística regularizada y
    el gradiente del costo w.r.t. a los parámetros.

    Parametros
    ----------
    theta : array_like
        Parámetro theta de la regresión logística. Vector de la forma (n, ). n es el número de características
        incluida la intercepción.

    X : array_like
        Dataset con la forma (m x n). m es el número de ejemplos, y n es el número de
        características (incluida la intercepción).

    y : array_like
        El conjunto de etiquetas. Un vector con la forma (m, ). m es el número de ejemplos.

    lambda_ : float
        Parámetro de regularización.

    Devuelve
    -------
    J : float
        El valor calculado para la función de costo regularizada.

    grad : array_like
        Un vector de la forma (n, ) que es el gradiente de la
        función de costo con respecto a theta, en los valores actuales de theta.
    """
    # Inicializa algunos valores útiles
    m = y.size

    # Calcula la hipótesis de la regresión logística
    h = sigmoid(X.dot(theta))

    # Calcula el costo sin regularización
    J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h)))

    # Calcula el término de regularización
    reg_term = (lambda_ / (2 * m)) * np.sum(theta[1:]**2)

    # Agrega el término de regularización al costo
    J += reg_term

    # Calcula el gradiente sin regularización
    grad = (1 / m) * X.T.dot(h - y)

    # Calcula el gradiente de la regularización
    grad[1:] += (lambda_ / m) * theta[1:]

    return J, grad

# Definir la función sigmoide
def sigmoid(z):
    return 1 / (1 + np.exp(-z))


#### 1.3.3 Vectorización regularizada de la regresión logística

Una vez implementada la vectorización para la regresión logística, corresponde agregarar regularización a la función de costo.



<a id="section2"></a>
### 1.4 Clasificacion One-vs-all
En esta parte del ejercicio, se implementará la clasificación de uno contra todos mediante el entrenamiento de múltiples clasificadores de regresión logística regularizados, uno para cada una de las clases en nuestro conjunto de datos.

In [50]:
import numpy as np
from scipy import optimize

def oneVsAll(X, y, num_labels, lambda_):
    """
    Entrena clasificadores de regresión logística uno contra todos y devuelve
    cada uno de estos clasificadores en una matriz all_theta, donde la fila i-ésima
    de all_theta corresponde al clasificador para la etiqueta i.

    Parámetros
    ----------
    X : array_like
        El conjunto de datos de entrada con forma (m x n). m es el número de
        puntos de datos, y n es el número de características. No asumimos
        que el término de intercepción (o sesgo) esté en X, sin embargo,
        proporcionamos el código a continuación para agregar el término de sesgo a X.

    y : array_like
        Las etiquetas de los datos. Un vector de forma (m, ).

    num_labels : int
        Número de etiquetas posibles.

    lambda_ : float
        Parámetro de regularización para la regresión logística.

    Retorna
    -------
    all_theta : array_like
        Los parámetros entrenados para la regresión logística para cada clase.
        Esta es una matriz de forma (K x n+1) donde K es el número de clases
        (es decir, `num_labels`) y n es el número de características sin el sesgo.
    """
    # Algunas variables útiles
    m, n = X.shape

    all_theta = np.zeros((num_labels, n + 1))

    # Agrega unos a la matriz X
    X = np.concatenate([np.ones((m, 1)), X], axis=1)

    for c in np.arange(num_labels):
        initial_theta = np.zeros(n + 1)
        options = {'maxiter': 50}
        res = optimize.minimize(lrCostFunction,
                                initial_theta,
                                (X, (y == c), lambda_),
                                jac=True,
                                method='CG',
                                options=options)

        all_theta[c] = res.x

    return all_theta


In [57]:
lambda_ = 0.1
all_theta = oneVsAll(X, y, num_labels, lambda_)
print(all_theta.shape)

(10, 16)


In [51]:
#Mi codigo con mis datos
lambda_ = 0.1
all_theta = oneVsAll(X_train, y_train, num_labels, lambda_)
print(all_theta.shape)


(10, 15)


In [52]:
print(all_theta)

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


<a id="section3"></a>
#### 1.4.1 Prediccion One-vs-all

Después de entrenar el clasificador de one-vs-all, se puede usarlo para predecir el dígito contenido en una imagen determinada. Para cada entrada, debe calcular la "probabilidad" de que pertenezca a cada clase utilizando los clasificadores de regresión logística entrenados. La función de predicción one-vs-all seleccionará la clase para la cual el clasificador de regresión logística correspondiente genera la probabilidad más alta y devolverá la etiqueta de clase (0, 1, ..., K-1) como la predicción para el ejemplo de entrada.

In [53]:
import numpy as np

def predictOneVsAll(all_theta, X):
    """
    Devuelve un vector de predicciones para cada ejemplo en la matriz X.
    Tenga en cuenta que X contiene los ejemplos en filas.
    all_theta es una matriz donde la i-ésima fila es un vector theta de regresión logística entrenada para la i-ésima clase.
    Debe establecer p en un vector de valores de 0..K-1 (por ejemplo, p = [0, 2, 0, 1]
    predice clases 0, 2, 0, 1 para 4 ejemplos).

    Parametros
    ----------
    all_theta : array_like
        The trained parameters for logistic regression for each class.
        This is a matrix of shape (K x n+1) where K is number of classes
        and n is number of features without the bias.

    X : array_like
        Data points to predict their labels. This is a matrix of shape
        (m x n) where m is number of data points to predict, and n is number
        of features without the bias term. Note we add the bias term for X in
        this function.

    Devuelve
    -------
    p : array_like
        The predictions for each data point in X. This is a vector of shape (m, ).
    """

    m = X.shape[0]
    num_labels = all_theta.shape[0]

    p = np.zeros(m)

    # Agrega unos a la matriz X
    X = np.concatenate([np.ones((m, 1)), X], axis=1)
    p = np.argmax(sigmoid(X.dot(all_theta.T)), axis=1)

    return p


Una vez que haya terminado, se llama a la función `predictOneVsAll` usando el valor aprendido de $\theta$. Debería apreciarse que la precisión del conjunto de entrenamiento es de aproximadamente 95,1% (es decir, clasifica correctamente el 95,1% de los ejemplos del conjunto de entrenamiento).

In [58]:
print(X.shape)
pred = predictOneVsAll(all_theta, X)
print('Precision del conjuto de entrenamiento: {:.2f}%'.format(np.mean(pred == y) * 100))
XPrueba = X[10:150, :].copy()
print(XPrueba.shape)
#print(np.ones((1)))
#print(XPrueba)
#p = np.zeros(1)
XPrueba = np.concatenate([np.ones((140, 1)), XPrueba], axis=1)
print(XPrueba.shape)
p = np.argmax(sigmoid(XPrueba.dot(all_theta.T)), axis = 1)
print(p)

# displayData(X[1002:1003, :])
print(y[10:150])

(17992, 15)
Precision del conjuto de entrenamiento: 30.94%
(140, 15)
(140, 16)
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
10     0.0
11     9.0
12     7.0
13     0.0
14     2.0
      ... 
145    0.0
146    0.0
147    8.0
148    9.0
149    0.0
Name: 5, Length: 140, dtype: float64
